# 🧪 PyTorch Lab 2: Newton Method for logistic regression


You already know **gradient descent**: update the parameters in the opposite direction of the gradient, with a fixed or adaptive step size.

**Newton’s method** goes a step further. Instead of using only the first derivative (the gradient), it also uses the **second derivative information** (the Hessian matrix). Around the current point, we approximate the loss by a quadratic function, then jump directly to its minimum.  

The update rule is:

$$
w_{k+1} = w_k - H^{-1}(w_k)\,\nabla f(w_k)
$$

where:  
- $ \nabla f(w_k)$ is the gradient at the current point,  
- $H(w_k)$ is the Hessian (matrix of second derivatives).  


We'll train on a small **real dataset** (Breast Cancer Wisconsin from scikit-learn). If scikit-learn isn't available, we fall back to a tiny embedded Iris-binary subset.

## 1) Setup


In [ ]:
import torch
torch.set_default_dtype(torch.float64)
print('PyTorch version:', torch.__version__)
torch.manual_seed(0)

import numpy as np
import matplotlib.pyplot as plt

# For data
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler


## 2) Load & prepare the data (Iris)


In [ ]:
# Load iris dataset
data = load_iris()
X = data['data']          # shape (150, 4)
y = data['target']        # classes: 0=setosa, 1=versicolor, 2=virginica

# Convert to binary classification: setosa (0) vs not-setosa (1)
y = (y != 0).astype(float)

# Standardize features
X = StandardScaler().fit_transform(X)

# Convert to torch tensors
X = torch.tensor(X, dtype=torch.get_default_dtype())
y = torch.tensor(y, dtype=torch.get_default_dtype())

n, d = X.shape

# Add bias column (intercept)
X = torch.cat([torch.ones(n, 1, dtype=X.dtype), X], dim=1)
d = d + 1
print(f"Tensors: X={tuple(X.shape)}, y={tuple(y.shape)}, d={d}")


## 3) Define the logistic loss

For a weight vector \(w \in \mathbb{R}^{d}\) and data \(X \in \mathbb{R}^{n\times d}\), labels \(y \in \{0,1\}^n\):

$$
\ell(w) = -\frac{1}{n}\sum_{i=1}^n \Big( y_i\log(\sigma(x_i^\top w)) + (1-y_i)\log(1-\sigma(x_i^\top w)) \Big),
$$

where $\sigma(t) = 1/(1+e^{-t})$ is the sigmoid.


In [ ]:
def logistic_loss(w, X, y):
    z = X @ w
    p = torch.sigmoid(z)
    eps = 1e-12
    nll = - (y * torch.log(p + eps) + (1 - y) * torch.log(1 - p + eps)).mean()
    return nll

# Initialize parameters
w = torch.nn.Parameter(torch.rand(X.shape[1], dtype=torch.get_default_dtype()), requires_grad=True)

# Quick sanity check
loss = logistic_loss(w, X, y)
print("Initial loss:", loss.item())


## 4) **Exercise:** Gradient & Hessian with `torch.autograd.functional`

**Tasks**
1. Compute the gradient of `logistic_loss` at `w`.
2. Compute the Hessian at `w`.
   
hint : torch.autograd.functional 


In [ ]:
# Fonction de perte logistique existante
def logistic_loss(w, X, y):
    z = X @ w
    p = torch.sigmoid(z)
    eps = 1e-12
    nll = -(y * torch.log(p + eps) + (1 - y) * torch.log(1 - p + eps)).mean()
    return nll

grad = torch.autograd.functional.grad(lambda w_: logistic_loss(w_, X, y), w)
H = torch.autograd.functional.hessian(lambda w_: logistic_loss(w_, X, y), w)

print("||grad||:", float(torch.norm(grad)))
print("H shape:", tuple(H.shape))

## 5) Gradient Descent (GD)

We implement a basic GD loop:
$$
w \leftarrow w - \alpha \nabla \ell(w).
$$

**Exercise**
- Complete the TODOs to record the loss at each epoch and update $w$.
- Use a stopping criterion based on $$ \|\nabla \ell(w)\| < \varepsilon $$


In [ ]:
n_epoch = 100
alpha = 1.0       # taux d'apprentissage
eps = 1e-2        # seuil de convergence pour la norme du gradient
loss_grad = []

# Réinitialisation des poids
w = torch.nn.Parameter(torch.rand(X.shape[1], dtype=torch.get_default_dtype()), requires_grad=True)

for epoch in range(n_epoch):
    # Calcul de la perte
    loss = logistic_loss(w, X, y)
    loss_grad.append(float(loss))
    
    # Calcul du gradient
    grad = torch.autograd.functional.grad(lambda w_: logistic_loss(w_, X, y), w)[0]
    
    # Mise à jour du vecteur poids w selon la règle GD
    with torch.no_grad():
        w -= alpha * grad
    
    # Critère d'arrêt basé sur la norme du gradient
    if torch.norm(grad) < eps:
        print(f"GD early stop at epoch {epoch}")
        break

print("Final loss:", loss_grad[-1])


## 6) Newton's Method

Newton update:
$$
w \leftarrow w - H(w)^{-1} \nabla \ell(w).
$$

In practice, **solve** the linear system `H s = grad` and set `w ← w − s` to avoid explicit inversion.


In [ ]:
n_epoch = 100
eps = 1e-2
loss_newton = []

# Réinitialiser w
w = torch.nn.Parameter(torch.rand(X.shape[1], dtype=torch.get_default_dtype()), requires_grad=True)

for epoch in range(n_epoch):
    # Calculer la perte
    loss = logistic_loss(w, X, y)
    loss_newton.append(float(loss))
    
    # Calculer gradient et Hessienne avec la fonction lambda
    grad = torch.autograd.functional.grad(lambda w_: logistic_loss(w_, X, y), w)[0]
    H = torch.autograd.functional.hessian(lambda w_: logistic_loss(w_, X, y), w)
    
    # Résoudre H * step = grad pour obtenir le pas de Newton
    step = torch.linalg.solve(H, grad)
    
    # Mise à jour de w avec le pas de Newton (sans trace dans autograd)
    with torch.no_grad():
        w -= step
    
    # Critère d'arrêt
    if torch.norm(grad) < eps:
        print(f"Newton early stop at epoch {epoch}")
        break

print("Final loss:", loss_newton[-1])

## 7) Compare convergence

Plot the loss trajectories of GD vs Newton.  


In [ ]:
import matplotlib.pyplot as plt

# Supposons loss_grad et loss_newton contiennent les historiques de pertes des deux méthodes

plt.figure(figsize=(8, 5))
plt.plot(loss_grad, label='Gradient Descent')
plt.plot(loss_newton, label='Newton Method')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Comparison of Loss Convergence')
plt.legend()
plt.grid(True)
plt.show()